# Imports

In [1]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.pela import Pela
from characters.fu_xuan import Fu_Xuan
from characters.dhil import DHIL
from characters.luocha import Luocha
from characters.tingyun import Tingyun
from utils.arena import Arena
from utils.data_table import Action_Table, Qingque_Probability_Table

# User Imputs

In [2]:
dhil_speed = 98

In [3]:
pela_speed = 161 

In [4]:
tingyun_speed = 161

In [5]:
luocha_speed = 161

In [6]:
sparkle_speed = 160.1

In [7]:
fu_xuan_speed = 134
fu_xuan_technique = True

# Simulations

## DHIL E0S0 Luocha Pela Sparkle

### Simulation

In [8]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [9]:
action_dataframe = Action_Table()

In [10]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)
            lch.basic(verbose=verbose)
            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1= False)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [11]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


### Results

In [12]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Luocha,0.0,62.111801,6,55.00,Basic,1
1,Pela,0.0,62.111801,7,55.00,Basic,1
2,Sparkle,0.0,62.460962,7,55.00,Skill,-1
3,Dan Heng IL,0.0,62.560962,6,70.00,Enhanced Basic Lvl 3,-3
4,Luocha,0.0,111.801242,3,78.88,Basic,1
5,Pela,0.0,124.223602,4,75.00,Basic,1
6,Sparkle,0.0,124.921924,5,90.82,Skill,-1
7,Sparkle,0.0,124.921924,4,126.64,Ultimate,4
8,Dan Heng IL,0.0,125.021924,7,110.00,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,150.00,Ultimate,0


In [13]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
3,Dan Heng IL,0.0,62.560962,6,70.0,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,7,110.0,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,150.0,Ultimate,0
13,Dan Heng IL,1.0,187.482886,5,5.0,Enhanced Basic Lvl 3,-1
17,Dan Heng IL,1.0,249.943848,5,45.0,Enhanced Basic Lvl 3,-3
23,Dan Heng IL,2.0,312.404809,7,85.0,Enhanced Basic Lvl 3,-3
27,Dan Heng IL,3.0,374.865771,5,125.0,Enhanced Basic Lvl 3,-3
28,Dan Heng IL,3.0,374.865771,2,165.0,Ultimate,0
32,Dan Heng IL,3.0,437.326733,3,5.0,Enhanced Basic Lvl 3,-1
37,Dan Heng IL,4.0,499.787695,7,45.0,Enhanced Basic Lvl 3,-3


## DHIL E0S1 Luocha Pela Sparkle

### Simulation

In [14]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed, s1 = True)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [15]:
action_dataframe = Action_Table()

In [16]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1 = True)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [17]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


### Results

In [18]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Luocha,0.0,62.111801,6,55.00,Basic,1
1,Pela,0.0,62.111801,6,55.00,Basic,1
2,Sparkle,0.0,62.460962,7,55.00,Skill,-1
3,Dan Heng IL,0.0,62.560962,6,70.00,Enhanced Basic Lvl 3,-3
4,Luocha,0.0,124.223602,3,55.00,Basic,1
5,Pela,0.0,124.223602,3,75.00,Basic,1
6,Sparkle,0.0,124.921924,4,90.82,Skill,-1
7,Sparkle,0.0,124.921924,3,126.64,Ultimate,4
8,Dan Heng IL,0.0,125.021924,7,114.80,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,159.60,Ultimate,0


In [19]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
3,Dan Heng IL,0.0,62.560962,6,70.0,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,7,114.8,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,159.6,Ultimate,0
13,Dan Heng IL,1.0,187.482886,4,5.0,Enhanced Basic Lvl 3,-1
17,Dan Heng IL,1.0,249.943848,3,49.8,Enhanced Basic Lvl 3,-3
22,Dan Heng IL,3.0,351.984664,6,94.6,Enhanced Basic Lvl 3,-3
26,Dan Heng IL,3.0,403.105072,3,139.4,Enhanced Basic Lvl 3,-3
27,Dan Heng IL,3.0,403.105072,0,184.2,Ultimate,0
31,Dan Heng IL,4.0,454.225480,0,5.0,Enhanced Basic Lvl 2,0
36,Dan Heng IL,5.0,556.266297,6,44.2,Enhanced Basic Lvl 3,-3


## DHIL E0S0 Luocha Tingyun Sparkle

### Simulation

In [20]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed)
  TY = Tingyun(arena = arena, recipient= DH, Spd = tingyun_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, TY, LCH

In [21]:
action_dataframe = Action_Table()

In [22]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, ty, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, ty, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif ty.action_gauge == 0:
        
        #Tingyun's actions
            if verbose:
                print("Tingyun takes a turn")
            ty.energy +=5*ty.energy_regen
            if arena.sp > 0 and ty.skill_counter <= 0:
                action_df.add_row(arena, ty, "Skill", -1)
                ty.skill(verbose=verbose)
            else:          
                action_df.add_row(arena, ty, "Basic", 1)
                ty.basic(verbose=verbose)
            if ty.max_energy <= ty.energy:
                action_df.add_row(arena, ty, "Ultimate", 0)
                ty.ult(verbose=verbose)
            ty.update_buffs()
            ty.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1= False)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [23]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


### Results

In [24]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
ty_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Tingyun"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Luocha,0.0,62.111801,6,55.000,Basic,1
1,Tingyun,0.0,62.111801,6,71.220,Skill,-1
2,Sparkle,0.0,62.460962,5,55.000,Skill,-1
3,Dan Heng IL,0.0,62.560962,4,70.000,Enhanced Basic Lvl 3,-3
4,Luocha,0.0,124.223602,1,55.000,Basic,1
5,Tingyun,0.0,124.223602,1,124.712,Basic,1
6,Tingyun,0.0,124.223602,2,159.544,Ultimate,0
7,Sparkle,0.0,124.921924,2,90.820,Skill,-1
8,Sparkle,0.0,124.921924,1,126.640,Ultimate,4
9,Dan Heng IL,0.0,125.021924,5,160.000,Ultimate,0


In [25]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
3,Dan Heng IL,0.0,62.560962,4,70.0,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,5,160.0,Ultimate,0
10,Dan Heng IL,0.0,125.021924,5,5.0,Enhanced Basic Lvl 3,-1
14,Dan Heng IL,1.0,187.482886,4,45.0,Enhanced Basic Lvl 3,-3
18,Dan Heng IL,2.0,289.523702,1,85.0,Enhanced Basic Lvl 1,-1
27,Dan Heng IL,3.0,374.865771,6,165.0,Ultimate,0
28,Dan Heng IL,3.0,374.865771,6,5.0,Enhanced Basic Lvl 3,-1
32,Dan Heng IL,3.0,437.326733,3,45.0,Enhanced Basic Lvl 3,-3
38,Dan Heng IL,4.0,539.367550,6,135.0,Enhanced Basic Lvl 3,-3
39,Dan Heng IL,4.0,539.367550,3,175.0,Ultimate,0


In [26]:
ty_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
1,Tingyun,0.0,62.111801,6,71.220,Skill,-1
5,Tingyun,0.0,124.223602,1,124.712,Basic,1
6,Tingyun,0.0,124.223602,2,159.544,Ultimate,0
12,Tingyun,1.0,186.335404,4,11.220,Basic,1
16,Tingyun,1.0,248.447205,1,52.272,Skill,-1
20,Tingyun,2.0,310.559006,0,105.764,Basic,1
21,Tingyun,2.0,310.559006,1,140.596,Ultimate,0
25,Tingyun,3.0,372.670807,6,11.220,Basic,1
30,Tingyun,3.0,434.782609,5,52.272,Skill,-1
34,Tingyun,4.0,496.894410,0,105.764,Basic,1


## DHIL E0S1 Luocha Pela Sparkle

In [27]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed, s1 = True)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [28]:
action_dataframe = Action_Table()

In [29]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1 = True)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [30]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [31]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Luocha,0.0,62.111801,6,55.00,Basic,1
1,Pela,0.0,62.111801,6,55.00,Basic,1
2,Sparkle,0.0,62.460962,7,55.00,Skill,-1
3,Dan Heng IL,0.0,62.560962,6,70.00,Enhanced Basic Lvl 3,-3
4,Luocha,0.0,124.223602,3,55.00,Basic,1
5,Pela,0.0,124.223602,3,75.00,Basic,1
6,Sparkle,0.0,124.921924,4,90.82,Skill,-1
7,Sparkle,0.0,124.921924,3,126.64,Ultimate,4
8,Dan Heng IL,0.0,125.021924,7,114.80,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,159.60,Ultimate,0


In [32]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
3,Dan Heng IL,0.0,62.560962,6,70.0,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,7,114.8,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,159.6,Ultimate,0
13,Dan Heng IL,1.0,187.482886,4,5.0,Enhanced Basic Lvl 3,-1
17,Dan Heng IL,1.0,249.943848,3,49.8,Enhanced Basic Lvl 3,-3
22,Dan Heng IL,3.0,351.984664,6,94.6,Enhanced Basic Lvl 3,-3
26,Dan Heng IL,3.0,403.105072,3,139.4,Enhanced Basic Lvl 3,-3
27,Dan Heng IL,3.0,403.105072,0,184.2,Ultimate,0
31,Dan Heng IL,4.0,454.225480,0,5.0,Enhanced Basic Lvl 2,0
36,Dan Heng IL,5.0,556.266297,6,44.2,Enhanced Basic Lvl 3,-3


## DHIL E0S0 Luocha Pela Sparkle

In [33]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [34]:
action_dataframe = Action_Table()

In [35]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1= False)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [36]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [37]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Luocha,0.0,62.111801,6,55.00,Basic,1
1,Pela,0.0,62.111801,6,55.00,Basic,1
2,Sparkle,0.0,62.460962,7,55.00,Skill,-1
3,Dan Heng IL,0.0,62.560962,6,70.00,Enhanced Basic Lvl 3,-3
4,Luocha,0.0,124.223602,3,55.00,Basic,1
5,Pela,0.0,124.223602,3,75.00,Basic,1
6,Sparkle,0.0,124.921924,4,90.82,Skill,-1
7,Sparkle,0.0,124.921924,3,126.64,Ultimate,4
8,Dan Heng IL,0.0,125.021924,7,110.00,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,150.00,Ultimate,0


In [38]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
3,Dan Heng IL,0.0,62.560962,6,70.0,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,7,110.0,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,150.0,Ultimate,0
13,Dan Heng IL,1.0,187.482886,4,5.0,Enhanced Basic Lvl 3,-1
17,Dan Heng IL,1.0,249.943848,3,45.0,Enhanced Basic Lvl 3,-3
22,Dan Heng IL,3.0,351.984664,6,85.0,Enhanced Basic Lvl 3,-3
26,Dan Heng IL,3.0,403.105072,3,125.0,Enhanced Basic Lvl 3,-3
27,Dan Heng IL,3.0,403.105072,0,165.0,Ultimate,0
31,Dan Heng IL,4.0,454.225480,0,5.0,Enhanced Basic Lvl 2,0
36,Dan Heng IL,5.0,556.266297,6,40.0,Enhanced Basic Lvl 3,-3


## DHIL E0S1 Luocha Pela Sparkle

In [39]:
def initialize_dhil():
  arena = Arena()
  DH = DHIL(arena = arena, Spd = dhil_speed, s1 = True)
  PL = Pela(arena = arena, Spd = pela_speed)
  LCH = Luocha(arena = arena, Spd = luocha_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, PL, LCH

In [40]:
action_dataframe = Action_Table()

In [41]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, pl, lch = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, pl, lch]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if lch.action_gauge == 0:
        
        #Luocha's actions
            if verbose:
                print("Luocha takes a turn")
            action_df.add_row(arena, lch, "Basic", 1)

            lch.update_buffs()
            lch.action_reset()

        elif pl.action_gauge == 0:
        
        #Pela's actions
            if verbose:
                print("Silver Wolf takes a turn")
            action_df.add_row(arena, pl, "Basic", 1)
            pl.basic(verbose=verbose)
            pl.update_buffs()
            pl.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0 and dh.skill_charges + arena.sp >1:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose, s1 = True)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [42]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [43]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Luocha,0.0,62.111801,6,55.00,Basic,1
1,Pela,0.0,62.111801,6,55.00,Basic,1
2,Sparkle,0.0,62.460962,7,55.00,Skill,-1
3,Dan Heng IL,0.0,62.560962,6,70.00,Enhanced Basic Lvl 3,-3
4,Luocha,0.0,124.223602,3,55.00,Basic,1
5,Pela,0.0,124.223602,3,75.00,Basic,1
6,Sparkle,0.0,124.921924,4,90.82,Skill,-1
7,Sparkle,0.0,124.921924,3,126.64,Ultimate,4
8,Dan Heng IL,0.0,125.021924,7,114.80,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,159.60,Ultimate,0


In [44]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
3,Dan Heng IL,0.0,62.560962,6,70.0,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,7,114.8,Enhanced Basic Lvl 3,-3
9,Dan Heng IL,0.0,125.021924,4,159.6,Ultimate,0
13,Dan Heng IL,1.0,187.482886,4,5.0,Enhanced Basic Lvl 3,-1
17,Dan Heng IL,1.0,249.943848,3,49.8,Enhanced Basic Lvl 3,-3
22,Dan Heng IL,3.0,351.984664,6,94.6,Enhanced Basic Lvl 3,-3
26,Dan Heng IL,3.0,403.105072,3,139.4,Enhanced Basic Lvl 3,-3
27,Dan Heng IL,3.0,403.105072,0,184.2,Ultimate,0
31,Dan Heng IL,4.0,454.225480,0,5.0,Enhanced Basic Lvl 2,0
36,Dan Heng IL,5.0,556.266297,6,44.2,Enhanced Basic Lvl 3,-3
